# Season 14 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [112]:
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [113]:
sim = SimulationData.load()
print(f"Last Updated Season {sim.season}, Day {sim.day}")

Last Updated Season 14, Day 9


---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decree will pass this season.

### Meal Plan
**All Fans will gain 2 new temporary Snack Slots each Earlsiesta and 2 new temporary Snack slots each Latesiesta. The 4
extra Slots and the Snacks in them will be washed away after each Election.**

### Pot Luck
**All Fans gain 3 Snack Slots permanently. Every Season, 3 Snacks in Concessions will be randomly selected to be
Spoiled, earning extremely low payouts, with a small chance of a Gourmet Jackpot.**

### All You Can Eat
**Fans will be able to freely buy and sell Snack Slots at any time. The less Snack Slots a Fan owns, the higher the
payout for the Snacks in them.**

All Decrees change how player Inventory is managed, by adding more item slots but having different downsides.

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swaping the Best Player on the Idol Board for
a position with our best player in that position).

**Best Case Swap:**

In [114]:
idols = [x.player for x in Idol.load().values()]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]


worst_batter = sort_lineup(pies, 1)[0]
print(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}")

pandas.DataFrame(replace_player(pies, worst_batter, bat_star=best_batting_idol.batting_stars, run_star=best_batting_idol.baserunning_stars, def_star=best_batting_idol.defense_stars))

Batting: Swap Jaxon Buckley for Yusef Puddles


,Pies
batting_change,+12.4348%
pitching_change,+0.0000%
baserunning_change,+3.3787%
defense_change,-1.7040%


In [115]:
worst_pitcher = sort_rotation(pies, 1)[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}")

pandas.DataFrame(replace_player(pies, worst_pitcher, pitch_star=best_pitching_idol.pitching_stars))


Pitching: Swap Nagomi Mcdaniel for Doc Anice


,Pies
batting_change,+0.0000%
pitching_change,+14.6353%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [116]:
best_batter = sort_lineup(pies, 1, order="best")[0]
print(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}")

pandas.DataFrame(replace_player(pies, best_batter, bat_star=best_batting_idol.batting_stars, run_star=best_batting_idol.baserunning_stars, def_star=best_batting_idol.defense_stars))

Batting: Swap Jaxon Buckley for Jaxon Buckley


,Pies
batting_change,+0.5191%
pitching_change,+0.0000%
baserunning_change,-0.5102%
defense_change,+0.0249%


In [117]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}")

pandas.DataFrame(replace_player(pies, best_pitcher, pitch_star=best_pitching_idol.pitching_stars))


Pitching: Swap Nagomi Mcdaniel for Elvis Figueroa


,Pies
batting_change,+0.0000%
pitching_change,+4.0673%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team.**

There are 139 possible Wild League players to swap with our 12 existing players, which is too many permuations for me to
list here. Instead, here's the list of the top 10 players available to choose from by star rating. For maximum impact it
is recommended we swap the player with the worst batter or pitcher respectively.

Note that position does not matter for the swap, so a player that is currently batting can still be added as a pitcher
and vice-versa.

**Best Batter options and Pies worst Batter:**

In [118]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)
    
all_wild.sort(key=lambda x: x.batting_rating, reverse=True)
set_heatmap(get_batting_stlats(all_wild[0:10]), maxVal=1)


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Nagomi Mcdaniel,1.249110,1.196404,1.345687,1.415884,1.479155,0.001000,1.060042,0.077418,0.878102
Beck Whitney,1.200078,0.991409,1.471090,1.268751,1.467907,0.001000,1.204729,0.001000,0.751079
Valentine Games,1.194971,1.087322,1.253784,1.682018,1.342306,0.001000,1.555499,0.019992,1.103348
Goodwin Morin,1.116151,1.011205,1.054999,1.804185,1.724130,0.001000,1.146978,0.054253,1.040647
Nagomi Nava,1.074517,0.736258,1.654306,0.761977,1.582309,0.111803,0.769464,0.001000,0.917821
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Vessalius Sundae,1.061020,0.887561,1.410623,0.561752,1.295647,0.001000,1.249409,0.001000,0.612723
Dudley Mueller,1.048644,1.565780,0.771321,0.945664,1.004410,0.223640,0.931322,0.060000,1.553180
Margarito Nava,1.039012,0.828340,1.284494,1.284454,1.444628,0.482716,1.171332,0.001000,0.826623
Gerund Pantheocide,1.027445,1.300367,1.034786,0.893386,0.575666,0.270360,0.571867,0.001000,0.289087


In [119]:
set_heatmap(get_batting_stlats(sort_lineup(pies, 1)), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


**Best Pitchers options and Pies worst Pitcher:**

In [120]:
all_wild.sort(key=lambda x: x.pitching_rating, reverse=True)
set_heatmap(get_pitching_stlats(all_wild[0:10]), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Nagomi Mcdaniel,1.337242,1.412762,1.087168,1.267917,1.482821,1.456696,1.078741
Burke Gonzales,1.176989,1.171639,1.341412,0.879956,0.870391,0.973638,0.789646
Qais Dogwalker,1.153869,1.071425,1.137700,1.358705,1.152111,0.882765,0.610121
Pitching Machine,1.133900,1.297717,1.186786,1.153428,0.443890,0.586661,0.832001
Goodwin Morin,1.022891,0.924709,0.912818,1.204162,1.809693,1.557388,1.288639
Lou Roseheart,0.961006,1.019976,0.835425,1.001713,1.215511,1.104658,1.076313
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
Hiroto Wilcox,0.929726,1.048886,1.108926,0.488923,0.771052,0.826826,0.337882
Valentine Games,0.892390,0.878882,0.988182,1.175546,0.627525,0.411364,0.943146
Rigby Friedrich,0.888575,1.072799,0.939847,0.427646,1.074809,0.718857,0.537667


In [121]:
set_heatmap(get_pitching_stlats(sort_rotation(pies, 1)), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Doc Anice,0.568234,0.928114,0.359045,0.836037,0.456020,0.598965,0.144698


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [122]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

get_stars(best_batting_shadows + [worst_batter])

,Batting,Pitching,Baserunning,Defense
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Jefferson Delacruz,3.0,0.5,3.5,4.0
Yusef Puddles,2.0,2.5,3.0,3.5


In [123]:
set_heatmap(get_batting_stlats(best_batting_shadows + [worst_batter]), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Jefferson Delacruz,0.595750,0.479940,0.643757,0.903986,0.537770,0.503675,0.434424,0.100000,0.671689
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


**Pitching:**

In [124]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

get_stars(best_pitching_shadows + [worst_pitcher])

,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,2.0,3.5,1.0,3.0
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0
Tiana Takahashi,1.0,3.0,2.5,3.0
Doc Anice,2.5,3.0,1.5,3.5


In [125]:
set_heatmap(get_pitching_stlats(best_pitching_shadows + [worst_pitcher]), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Doc Anice,0.568234,0.928114,0.359045,0.836037,0.456020,0.598965,0.144698


### Infuse
**Boost a Player on your Team by 15% to 30%.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

**15% Change:**

In [126]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,3.068746,0.920935
Beasley Day,2.695709,3.559653,0.863944
Lang Richardson,3.162214,3.907698,0.745485
Huber Frumple,3.806051,4.575154,0.769103
Jode Preston,4.021708,4.773353,0.751645
Eduardo Woodman,4.165300,4.885272,0.719972
Jaxon Buckley,6.318317,6.956462,0.638145


In [127]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.0)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Beasley Day,0.711931,0.835000,0.838801,0.227669,0.358479,0.318964,0.673706,0.010000,0.903672
Lang Richardson,0.781540,0.962722,0.550621,0.772843,1.091821,0.055246,0.648400,0.010000,0.701015
Huber Frumple,0.915031,0.573284,1.319818,1.027497,1.347390,0.195580,0.794357,0.010000,1.213890
Jode Preston,0.954671,1.088336,1.094763,0.720218,0.334025,0.010000,0.987760,0.010000,0.280957
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Jaxon Buckley,1.391292,1.643269,1.137449,1.826092,2.002606,0.010000,2.082640,0.010000,1.477099


In [128]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Doc Anice,2.841172,3.734213,0.893041
Henry Marshallow,3.439741,4.374069,0.934328
Ruslan Greatness,4.096695,4.976003,0.879308
Nerd Pacheco,4.180600,5.151709,0.971109
Elvis Figueroa,5.483164,6.399624,0.916460


In [129]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1)


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Doc Anice,0.746843,1.078114,0.509045,0.986037,0.606020,0.748965,0.294698
Henry Marshallow,0.874814,0.573292,1.256023,1.531125,0.924138,0.895639,0.971668
Ruslan Greatness,0.995201,0.920870,0.936481,1.419963,1.176832,0.779742,1.570762
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Elvis Figueroa,1.279925,1.309675,1.286143,1.222682,0.936089,0.611405,0.871203


**30% Change:**

In [130]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,3.885398,1.737587
Beasley Day,2.695709,4.319837,1.624128
Lang Richardson,3.162214,4.606233,1.444019
Huber Frumple,3.806051,5.311149,1.505098
Jode Preston,4.021708,5.463113,1.441405
Eduardo Woodman,4.165300,5.585918,1.420618
Jaxon Buckley,6.318317,7.586190,1.267873


In [131]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Beasley Day,0.863967,0.985000,0.988801,0.377669,0.508479,0.168964,0.823706,0.010000,1.053672
Lang Richardson,0.921247,1.112722,0.700621,0.922843,1.241821,0.010000,0.798400,0.010000,0.851015
Huber Frumple,1.062230,0.723284,1.469818,1.177497,1.497390,0.045580,0.944357,0.010000,1.363890
Jode Preston,1.092623,1.238336,1.244763,0.870218,0.484025,0.010000,1.137760,0.010000,0.430957
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Jaxon Buckley,1.517238,1.793269,1.287449,1.976092,2.152606,0.010000,2.232640,0.010000,1.627099


In [132]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Doc Anice,2.841172,4.636189,1.795017
Henry Marshallow,3.439741,5.311955,1.872214
Ruslan Greatness,4.096695,5.876536,1.779841
Nerd Pacheco,4.180600,6.110125,1.929525
Elvis Figueroa,5.483164,7.332289,1.849125


In [133]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Doc Anice,0.927238,1.228114,0.659045,1.136037,0.756020,0.898965,0.444698
Henry Marshallow,1.062391,0.723292,1.406023,1.681125,1.074138,1.045639,1.121668
Ruslan Greatness,1.175307,1.070870,1.086481,1.569963,1.326832,0.929742,1.720762
Nerd Pacheco,1.222025,1.387720,1.366351,0.572113,0.908134,1.232212,0.784500
Elvis Figueroa,1.466458,1.459675,1.436143,1.372682,1.086089,0.761405,1.021203


### Trust
**Receive a mystery Benefit for one of your Teams' Players.**

The Benifit here is unknown. It is likely either a stat increase or an unknown Modification.

### Alternate
**Call in an Alternate for a Player on your Team.**

Alternates are when players stats are randomized. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [134]:
worst_pie = sort_overall(pies, 1)[0]

print(f"Rerolling {worst_pie.name}")
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling Yusef Puddles


,Pies
batting_change,-0.4223%
pitching_change,+0.0000%
baserunning_change,-0.9070%
defense_change,-3.1326%


### Revoke
**Revoke a Player from your roster.**

Revoking will cause the player to change teams, potentially every season. This will cause our other players to play more
often, increasing our average team performance if we chose the worst player. See below for analysis of removing our
worst Batter or Pitcher.

**Batting:**

In [135]:
print(f'Removing {worst_batter.name}')
pandas.DataFrame(remove_player(pies, worst_batter))

Removing Yusef Puddles


,Pies
batting_change,+5.3726%
pitching_change,+0.0000%
baserunning_change,+2.5066%
defense_change,-1.8743%


**Pitching:**

In [136]:
print(f'Removing {worst_pitcher.name}')
pandas.DataFrame(remove_player(pies, worst_pitcher))


Removing Doc Anice


,Pies
batting_change,+0.0000%
pitching_change,+5.8355%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* Shame Donor: A Team or Player who adds Unruns to a Shamed opponents' future pile.
* Earlybirds: This team will Overperform in the Earlseason.
* Late to the Party: This team will Overperform in the Lateseason.

Seasonal Modifications have been mixed in the past. For Fourth Strike Analysis, see the appropriate
[SIBR Paper](https://research.blaseball-reference.com/Fourth_Strike.pdf). Earlybirds or Late to the Party would be
roughly equivalent to a 5% boost overall.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Jam-Packed & Glazed
**Your Team will pay out double to all Fans when they trigger the Black Hole.**

**Your Team will pay out double to all Fans when they trigger Sun 2.**

See below for analysis of likelihood to trigger Sun 2 & Black Hole, as well as the estimated payout difference.

In [137]:
pies_games = Game.load_by_season(season=13, team_id=pies.id)
sun2_games = [x for x in pies_games.values() if f"Sun 2 set a Win upon the {pies.nickname}" in x.outcomes or f"Sun 2 set a Win upon the {pies.nickname}." in x.outcomes]
total_sun2 = len(sun2_games)
print(f"Sun 2 was activated for the {pies.nickname} {total_sun2} times in Season 13")

Sun 2 was activated for the Pies 0 times in Season 13


In [138]:
total_blackholes = 0
for game in pies_games.values():
    for outcome in game.outcomes:
        if "The Black Hole swallowed a Win" in outcome and pies.nickname not in outcome:
            total_blackholes += 1
print(f"Black Hole was activated for the {pies.nickname} {total_blackholes} times in Season 13.")

Black Hole was activated for the Pies 1 times in Season 13.


In [139]:
print(f"With Max Wet Pretzels, win payout difference: {15000*total_blackholes} vs {15000*total_blackholes*2}")
print(f"With Max Doughnuts, loss payout difference: {10000*total_sun2} vs {10000*total_sun2*2}")


With Max Wet Pretzels, win payout difference: 15000 vs 30000
With Max Doughnuts, loss payout difference: 0 vs 0


### Buttered Up & Over Salted
**Your Team will pay out double for Popcorn.**

**Your Team will pay out double for Stale Popcorn.**

See below for analysis of Seasonal wins and losses, as well as the estimated payout difference.


In [140]:
wins = [x for x in pies_games.values() if x.winning_team == pies]
losses = [x for x in pies_games.values() if x.winning_team != pies]
print(f"Pies won {len(wins)} games and lost {len(losses)} games in Season 13")
print(f"With Max Popcorn, win payout difference: {300*len(wins)} vs {300*len(wins)*2}")
print(f"With Max Stale Popcorn, loss payout difference: {800*len(losses)} vs {800*len(losses)*2}")


Pies won 43 games and lost 56 games in Season 13
With Max Popcorn, win payout difference: 12900 vs 25800
With Max Stale Popcorn, loss payout difference: 44800 vs 89600


### Solo Stroll
**The worst Player on your Team will draw a walk on 3 Balls instead of 4.**

Assuming this means worst *Batter* on the team, see below for player details. Note that `moxie` is that attribute most
correlated to drawing walks.

In [141]:
get_batting_stlats(worst_batter)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


In [142]:
get_batting_stats(worst_batter, 13)

,appearances,batting_average,on_base_percentage,slugging,plate_appearances,at_bats,hits,walks,singles,doubles,triples,quadruples,home_runs,runs_batted_in,strikeouts,sacrifice_bunts,sacrifice_flies,at_bats_risp,hits_risp,batting_average_risp,on_base_slugging,total_bases,hit_by_pitches,ground_outs,flyouts,gidp
Yusef Puddles,98,0.205,0.229,0.331,508,483,99,16,64,19,6,0,10,46,147,3,0,77,22,0.286,0.560,160,0,121,94,2


### Min-Maxing
**Every Team in your division gets one Minimalist and one Maximalist.**

"Minimalist" and "Maximalist" are unknown.

### Targeted Evolution
**A random pitcher and a random hitter on your Team will Advance.**

Evolving or Advancing will give the player an additional Gold Star in all stats. This gold star will prevent debuffs
from dropping the player below that star rating.

### The Nut Button
**Your Team is no longer allergic to Peanuts.**

See below for peanut allergy stats for the team. The Crab's Peanut Mister can also cure peanut allergies when playing in
the Crabitat.

In [143]:
peanut_stats = [x.peanut_allergy for x in pies.lineup + pies.rotation]
peanut_percent = peanut_stats.count(True)/len(peanut_stats)
print(f"Pies are {peanut_percent*100}% Allergic.")

pandas.DataFrame([{"Allergic": x} for x in peanut_stats], index=[x.name for x in pies.lineup + pies.rotation])

Pies are 50.0% Allergic.


,Allergic
Yusef Puddles,True
Beasley Day,True
Lang Richardson,False
Huber Frumple,True
Jode Preston,True
Eduardo Woodman,False
Jaxon Buckley,True
Doc Anice,False
Henry Marshallow,False
Ruslan Greatness,False


### Non-Dominant Arms
**Maybe try throwing with the other one? Your Team's pitching changes anywhere from -5% to +15%.**

See below for analysis of the best and worst case rolls on our pitching.

**Worst Case (-5%)**

In [144]:
table, _, _ = improve_team_pitching_table(pies, -0.05)
table

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Doc Anice,2.841172,2.543396,-0.297776
Henry Marshallow,3.439741,3.127075,-0.312666
Ruslan Greatness,4.096695,3.808712,-0.287984
Nerd Pacheco,4.180600,3.847040,-0.333561
Elvis Figueroa,5.483164,5.181238,-0.301926


In [145]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, -0.05)), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Doc Anice,0.508679,0.878114,0.309045,0.786037,0.406020,0.548965,0.094698
Henry Marshallow,0.625415,0.373292,1.056023,1.331125,0.724138,0.695639,0.771668
Ruslan Greatness,0.761742,0.720870,0.736481,1.219963,0.976832,0.579742,1.370762
Nerd Pacheco,0.769408,1.037720,1.016351,0.222113,0.558134,0.882212,0.434500
Elvis Figueroa,1.036248,1.109675,1.086143,1.022682,0.736089,0.411405,0.671203


**Best Case (15%)**

In [146]:
table, _, _ = improve_team_pitching_table(pies, 0.15)
table

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Doc Anice,2.841172,3.734213,0.893041
Henry Marshallow,3.439741,4.374069,0.934328
Ruslan Greatness,4.096695,4.976003,0.879308
Nerd Pacheco,4.180600,5.151709,0.971109
Elvis Figueroa,5.483164,6.399624,0.916460


In [147]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1)


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Doc Anice,0.746843,1.078114,0.509045,0.986037,0.606020,0.748965,0.294698
Henry Marshallow,0.874814,0.573292,1.256023,1.531125,0.924138,0.895639,0.971668
Ruslan Greatness,0.995201,0.920870,0.936481,1.419963,1.176832,0.779742,1.570762
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Elvis Figueroa,1.279925,1.309675,1.286143,1.222682,0.936089,0.611405,0.871203


### Precognition
**Vision of things to come. Improve 3 random hitters on your Team by 20%.**

See below for analysis of how +20% would affect our batters.


In [148]:
table, _, _ = improve_team_batting_table(pies, 0.20)
table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,3.348481,1.200670
Beasley Day,2.695709,3.818814,1.123105
Lang Richardson,3.162214,4.149450,0.987236
Huber Frumple,3.806051,4.823281,1.017230
Jode Preston,4.021708,5.007238,0.985530
Eduardo Woodman,4.165300,5.120266,0.954966
Jaxon Buckley,6.318317,7.167583,0.849266


In [149]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.20)), maxVal=1)


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.669696,1.233698,0.384577,0.875196,0.351908,0.430778,0.310821,0.010000,1.210015
Beasley Day,0.763763,0.885000,0.888801,0.277669,0.408479,0.268964,0.723706,0.010000,0.953672
Lang Richardson,0.829890,1.012722,0.600621,0.822843,1.141821,0.010000,0.698400,0.010000,0.751015
Huber Frumple,0.964656,0.623284,1.369818,1.077497,1.397390,0.145580,0.844357,0.010000,1.263890
Jode Preston,1.001448,1.138336,1.144763,0.770218,0.384025,0.010000,1.037760,0.010000,0.330957
Eduardo Woodman,1.024053,1.219391,1.046540,0.850666,0.775519,0.395517,0.774554,0.010000,0.905869
Jaxon Buckley,1.433517,1.693269,1.187449,1.876092,2.052606,0.010000,2.132640,0.010000,1.527099


### Headliners
**Arrange your lineup in order of their Idolatry.**

The exact ordering of our team by Idolatry is unknown.

### Soul Swap
**They won't stop screaming. Randomizes your Team's 5 worst players.**

This will attempt to reroll the following players, in order. Players are rerolled in order, so if a player is rerolled
to be worse they will be rerolled again, as they are still the worst pitcher.
For reference, the Statistical Average player has 2 batting stars and 1.5 pitching stars.

In [150]:
worst_players = sort_overall(pies, 5)
get_stars(worst_players)

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Beasley Day,2.5,1.5,2.5,1.5
Doc Anice,2.5,3.0,1.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Henry Marshallow,3.5,3.5,2.5,4.5


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
